# GANs (Generative Adversarial Networks)

## Abstract

Our objective is to learn how to implement Generative Adversarial Networks(GANs) using Keras and TensorFlow. We will be implementing on Fashion MNIST dataset.

**Objective**
1. Discuss Generative Adversarial Networks
2. Implement GAN architecture using Keras and Tensorflow
3. Train Fashion MNIST dataset
4. Generate fake/synthetic apparel images



**Generative Adversarial Networks** can be used to generate synthetic (i.e., fake) images that are perceptually near identical to their ground-truth authentic originals.

<Img src="https://github.com/rhnyewale/INFO-7390-Advance-Data-Science-Architecture/blob/master/Mini_Project3-GAN/Images/GANs.jpg?raw=true">

In order to generate synthetic images, we make use of two neural networks during training:

1. A generator that accepts an input vector of randomly generated noise and produces an output “imitation” image that looks similar, if not identical, to the authentic image
2. A discriminator or adversary that attempts to determine if a given image is an “authentic” or “fake”

By training these networks at the same time, one giving feedback to the other, we can learn to generate synthetic images.

## General GAN training procedure

<Img src="https://github.com/rhnyewale/INFO-7390-Advance-Data-Science-Architecture/blob/master/Mini_Project3-GAN/Images/GANs_Training%20.jpg?raw=true">
    
    
**Step 1** we randomly generate a vector (i.e., noise). We pass this noise through our generator,
    
**Step 2** which generates an actual image in **Step 2**
    
**Step 3** We then sample authentic images from our training set and mix them with our synthetic images

**Step 4** next step is to train our discriminator using this mixed set. The goal of the discriminator is to correctly label each image as “real” or “fake.”

**Step 5** Next, we’ll once again generate random noise, but this time we’ll purposely label each noise vector as a “real image” 

**Step 6** We’ll then train the GAN using the noise vectors and “real image” labels even though they are not actual real images
    
    
**The reason this process works is due to the following:**

1. We have frozen the weights of the discriminator at this stage, implying that the discriminator is not learning when we update the weights of the generator.
    <br/><br/>
2. We’re trying to “fool” the discriminator into being unable to determine which images are real vs. synthetic. The feedback from the discriminator will allow the generator to learn how to produce more authentic images.

In [2]:
!pip install tensorflow==2.2.0

In [4]:
pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25856 sha256=123664ff0e1e585427416b893f5d07bad40e7a2445bdb2fb0ccde3eaa61c3860
  Stored in directory: c:\users\rhnye\appdata\local\pip\cache\wheels\fc\9c\6d\1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from matplotlib import pyplot as plt
from imutils import build_montages
from sklearn.utils import shuffle
import numpy as np
import argparse
import cv2
import os

 **Function to display images in Jupyter Notebooks and Google Colab**

In [4]:
def plt_imshow(title, image):
  # convert the image frame BGR to RGB color space and display it
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image, aspect="auto")
    plt.title(title)
    plt.grid(False)
    plt.show()

**Implementing our “generator” and “discriminator” with Keras and TensorFlow**

In [6]:
class DCGAN:
    @staticmethod
    def build_generator(dim, depth, channels=1, inputDim=100,
        outputDim=512):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (dim, dim, depth)
        chanDim = -1

        # first set of FC => RELU => BN layers
        model.add(Dense(input_dim=inputDim, units=outputDim))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        # second set of FC => RELU => BN layers, this time preparing
        # the number of FC nodes to be reshaped into a volume
        model.add(Dense(dim * dim * depth))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
  
        # reshape the output of the previous layer set, upsample +
        # apply a transposed convolution, RELU, and BN
        model.add(Reshape(inputShape))
        model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
            padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
  
        # apply another upsample and transposed convolution, but
        # this time output the TANH activation
        model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
            padding="same"))
        model.add(Activation("tanh"))

        # return the generator model
        return model

    @staticmethod
    def build_discriminator(width, height, depth, alpha=0.2):
        # initialize the model along with the input shape to be
        # "channels last"
        model = Sequential()
        inputShape = (height, width, depth)

        # first set of CONV => RELU layers
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
            input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))

        # second set of CONV => RELU layers
        model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=alpha))

        # sigmoid layer outputting a single value
        model.add(Dense(1))
        model.add(Activation("sigmoid"))

        # return the discriminator model
        return model

**Implementing our GAN training script

In [7]:
args = {
    "output": "output",
    "epochs": 50,
    "batch_size": 128
}

In [8]:
# store the epochs and batch size in convenience variables, then
# initialize our learning rate
NUM_EPOCHS = args["epochs"]
BATCH_SIZE = args["batch_size"]
INIT_LR = 2e-4

In [9]:
# load the Fashion MNIST dataset and stack the training and testing
# data points so we have additional training data
print("[INFO] loading MNIST dataset...")
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])

# add in an extra dimension for the channel and scale the images
# into the range [-1, 1] (which is the range of the tanh
# function)
trainImages = np.expand_dims(trainImages, axis=-1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5

[INFO] loading MNIST dataset...
4423680/4422102 [==============================] - 0s 0us/step


In [10]:
# build the generator
print("[INFO] building generator...")
gen = DCGAN.build_generator(7, 64, channels=1)

# build the discriminator
print("[INFO] building discriminator...")
disc = DCGAN.build_discriminator(28, 28, 1)
discOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building generator...
[INFO] building discriminator...


# Citation

https://arxiv.org/abs/1511.06434<br/>



# License

Copyright 2020 Rohan Subhash Yewale

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.